<a href="https://colab.research.google.com/github/Nick088Official/XTTS2_UI_Inference_Training_Google_Colab/blob/main/XTTS2_Inf_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset building + XTTS2 finetuning and inference

### **Install**
Run the first 3 cells (Allow Google Colab to use your Google Drive Account to import here the finished model, ignore pip install errors in the first one)

Then click on the link `Running on public URL: ` when the demo is ready.


### **Download the model & processed dataset**

Run the last 3 cells


### **Import a model & processed dataset**
Run the first 3 cells (Allow Google Colab to use your Google Drive Account to import here the model you wanna use, so be sure you uploaded the model (.pth, config.json, vocab.json, any speaker audio refernce from the dataset), ignore pip install errors in the first one)

Then click on the link `Running on public URL: ` when the demo is ready.

Skip to Inference

Put "/content/drive/MyDrive/XTTS2_Inf_Train/model.pth" in the checkpoint path section

Put "/content/drive/MyDrive/XTTS2_Inf_Train/config.json" in the configh path section

Put "/content/drive/MyDrive/XTTS2_Inf_Train/vocab.json" in the vocab path section

Put "/content/drive/MyDrive/XTTS2_Inf_Train/(name of any of the .wav inside of the processed dataset, that needs to be in the Google Drive Folder too)" in the speaker reference audio section (XTTS2 will generate different types of quality output based on which audio you choose)

NOTE: If the model is in a folder under the Google Drive XTTS_ft_colab folder, you will have to put that too in all sections, for example: "/content/drive/MyDrive/XTTS2_Inf_Train/test/model.pth"

Load Fine Tuned Model, input any text and click on Inference


You can also watch this guide https://www.youtube.com/watch?v=8tpDiiouGxc , but its from their original colab which is a bit different from this

In [ ]:
#@title Install XTTS2
!rm -rf TTS/ # delete repo to be able to reinstall if needed
!git clone --branch xtts_demo -q https://github.com/coqui-ai/TTS.git
!pip install --use-deprecated=legacy-resolver -q -e TTS
!pip install --use-deprecated=legacy-resolver -q -r TTS/TTS/demos/xtts_ft_demo/requirements.txt
!pip install -q typing_extensions==4.8 numpy==1.26.2

In [ ]:
#@title Mount Google Drive
from google.colab import drive
import shutil
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/XTTS2_Inf_Train/

In [ ]:
#@title Run UI
!python TTS/TTS/demos/xtts_ft_demo/xtts_demo.py

In [ ]:
#@title Download the Procesed Dataset (Audio Speaker Reference) of the finetuned model
from google.colab import files

!zip -q -r dataset.zip /tmp/xtts_ft/dataset
files.download('dataset.zip')

In [ ]:
#@title Download the finetuned Model (.pth, config.json, vocab.json)
from google.colab import files
import os
import glob
import torch

def find_latest_best_model(folder_path):
    search_path = os.path.join(folder_path, '**', 'best_model.pth')
    files = glob.glob(search_path, recursive=True)
    latest_file = max(files, key=os.path.getctime, default=None)
    return latest_file

model_path = find_latest_best_model("/tmp/xtts_ft/run/training/")
checkpoint = torch.load(model_path, map_location=torch.device("cpu"))
del checkpoint["optimizer"]
for key in list(checkpoint["model"].keys()):
    if "dvae" in key:
        del checkpoint["model"][key]
torch.save(checkpoint, "model.pth")
model_dir = os.path.dirname(model_path)
files.download(os.path.join(model_dir, 'config.json'))
files.download(os.path.join(model_dir, 'vocab.json'))
files.download('model.pth')

In [ ]:
#@title Download the Dataset & Model in Google Drive
#@markdown The two previous cells are a requirement for this step but it can be much faster
from google.colab import drive
import shutil
drive.mount('/content/drive')

model_name = "test" #@param {type:"string"}
give_model_name = True #@param {type:"boolean"}

if give_model_name:
  # Create the directory for the model in Google Drive
  model_folder = "/content/drive/MyDrive/XTTS2_Inf_Train/" + model_name
  !mkdir -p {model_folder}
  shutil.copy(os.path.join(model_dir, 'config.json'), model_folder)
  shutil.copy(os.path.join(model_dir, 'vocab.json'), model_folder)
  shutil.copy('model.pth', model_folder)
  shutil.copy('dataset.zip', model_folder)
else:
  !mkdir /content/drive/MyDrive/XTTS_ft_colab
  shutil.copy(os.path.join(model_dir, 'config.json'), "/content/drive/MyDrive/XTTS2_Inf_Train/config.json")
  shutil.copy(os.path.join(model_dir, 'vocab.json'), "/content/drive/MyDrive/XTTS2_Inf_Train/vocab.json'")
  shutil.copy('model.pth', "/content/drive/MyDrive/XTTS2_Inf_Train/model.pth")
  shutil.copy('dataset.zip', "/content/drive/MyDrive/XTTS2_Inf_Train/dataset.zip")